In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima
import db_connection as dbcon

#%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# parameters
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')

# connect to db
db = dbcon.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

DB Credentials from file
DB connected successfully!!!


In [65]:
query = {
    "$and" : [
        { "name":"Barcelona" },
        #{"gist": {"$exists": True} },
        {"gist": {"$exists": False} }
    ]  
}
images_metadata = [img_metadata for img_metadata in images_lib_col.find(query)]
print(len(images_metadata))

0


In [66]:
for image in images_metadata:
    print(image["filename"])

In [67]:
images_filenames = [image["filename"] for image in images_metadata]
images_files = ima.load_images_from_gdrive(images_filenames, IMG_FOLDER, return_list=True)
len(images_files)

0

In [68]:
import gist

feature_vectors = []
for image in images_files:
    print(image["fname"])
    print(image["array"].shape)
    image["gist"] = gist.extract(image["array"])
    feature_vectors.append(image["gist"])
    print(image["gist"][:10])

In [69]:
for image in images_files:
    result = images_lib_col.update(
        {"filename": image["fname"]},
        { "$set": {"gist" : image["gist"].tolist()} }
    )
    print(result["nModified"])